# Print Test IoU after final incremental step for multistep runs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import wandb
import os
import json
os.chdir("/home/thesis/marx/wilson_gen")
import sys
sys.path.insert(0, "/home/thesis/marx/wilson_gen/WILSON")
from WILSON import tasks

wandb.login()
api = wandb.Api()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: gooogleon (gooogleon-uni-hd). Use `wandb login --relogin` to force relogin


In [2]:
RUN_NAMES = [
    "Incr_Gen_Lora_RR",
    "Incr_Gen_Lora_OD_Inp_RR",
    "Incr_Gen_Base_RR",
    "Incr_Gen_Base_OD_Inp_RR",
]

TASKS = [
    "10-10",
    "15-5",
    "10-5",
    "10-1",
    "15-1",
]

In [6]:
runs = api.runs("gooogleon-uni-hd/WILSON")
runs = [run for run in runs if "final_exp_1" in run.tags]
print(runs)

[<Run gooogleon-uni-hd/WILSON/321lfj19 (finished)>, <Run gooogleon-uni-hd/WILSON/0y6hz3f9 (finished)>, <Run gooogleon-uni-hd/WILSON/h34pozkj (finished)>, <Run gooogleon-uni-hd/WILSON/srihq4e7 (finished)>, <Run gooogleon-uni-hd/WILSON/bpf1qeva (finished)>, <Run gooogleon-uni-hd/WILSON/k0yx9c14 (finished)>, <Run gooogleon-uni-hd/WILSON/pklazz7m (finished)>, <Run gooogleon-uni-hd/WILSON/dw82d5bd (finished)>, <Run gooogleon-uni-hd/WILSON/ry98p9l4 (finished)>, <Run gooogleon-uni-hd/WILSON/8n2o5q3q (finished)>, <Run gooogleon-uni-hd/WILSON/9p2wnktp (finished)>, <Run gooogleon-uni-hd/WILSON/n9ijm804 (finished)>, <Run gooogleon-uni-hd/WILSON/3zamt3yy (finished)>, <Run gooogleon-uni-hd/WILSON/67szcjnp (finished)>, <Run gooogleon-uni-hd/WILSON/9fno9d8y (finished)>, <Run gooogleon-uni-hd/WILSON/duhct6i3 (finished)>, <Run gooogleon-uni-hd/WILSON/utrj61uc (finished)>, <Run gooogleon-uni-hd/WILSON/970oelmf (finished)>, <Run gooogleon-uni-hd/WILSON/c8k73jkz (finished)>, <Run gooogleon-uni-hd/WILSON/v

In [12]:
runs_and_values = {}
for run in runs:
    if run.state == "finished":
        final_miou = run.summary["Final/MeanIoU_All"]
        dense_miou = run.summary["Final/MeanIoU_Dense"]
        incr_miou = run.summary["Final/MeanIoU_Incr"]
        if final_miou == "X":
            continue
        else:
            ov_string = "ov" if "ov" in run.name else "dj"
            run_name = run.name.replace("-ov", "")
            if run_name not in runs_and_values:
                runs_and_values[run_name] = {
                    ov_string: {
                        "final_miou": final_miou,
                        "dense_miou": dense_miou,
                        "incr_miou": incr_miou,
                    }
                }
            else:
                if ov_string not in runs_and_values[run_name]:
                    runs_and_values[run_name][ov_string] = {
                        "final_miou": final_miou,
                        "dense_miou": dense_miou,
                        "incr_miou": incr_miou,
                    }
                else:
                    print("Warning: Run already exists in runs_and_values")
    else:
        raise ValueError("Final mIoU should not be X")

In [21]:
for task in TASKS:
    print(task)
    for rn in RUN_NAMES:
        key = f"voc-{task}_{rn}"
        rn = rn.replace("_RR", "")
        rn = rn.replace("Incr_Gen_", "")
        rn = rn.replace("_OD_Inp", " (OD_INP)")
        # print(rn)
        # print(key)
        to_print = f"                        Ours: {rn} & "
        to_print += f"{np.round(100 * runs_and_values[key]['dj']['dense_miou'], 1)} & "
        to_print += f"{np.round(100 * runs_and_values[key]['dj']['incr_miou'], 1)} & "
        to_print += f"{np.round(100 * runs_and_values[key]['dj']['final_miou'], 1)} & "
        to_print += f"{np.round(100 * runs_and_values[key]['ov']['dense_miou'], 1)} & "
        to_print += f"{np.round(100 * runs_and_values[key]['ov']['incr_miou'], 1)} & "
        to_print += f"{np.round(100 * runs_and_values[key]['ov']['final_miou'], 1)} \\\\"
        print(to_print)

10-10
                        Ours: Lora & 67.1 & 52.3 & 61.1 & 72.1 & 56.9 & 65.7 \\
                        Ours: Lora (OD_INP) & 65.9 & 49.1 & 59.0 & 72.2 & 56.9 & 65.7 \\
                        Ours: Base & 67.3 & 52.4 & 61.2 & 71.6 & 56.8 & 65.4 \\
                        Ours: Base (OD_INP) & 65.9 & 49.0 & 59.0 & 71.8 & 56.7 & 65.4 \\
15-5
                        Ours: Lora & 77.2 & 45.9 & 70.3 & 77.8 & 46.9 & 71.0 \\
                        Ours: Lora (OD_INP) & 76.6 & 41.4 & 68.8 & 78.0 & 44.4 & 70.5 \\
                        Ours: Base & 77.0 & 45.4 & 70.1 & 77.6 & 46.4 & 70.8 \\
                        Ours: Base (OD_INP) & 76.5 & 42.6 & 69.1 & 77.7 & 44.5 & 70.3 \\
10-5
                        Ours: Lora & 65.6 & 46.2 & 57.4 & 69.8 & 50.9 & 61.6 \\
                        Ours: Lora (OD_INP) & 64.6 & 41.2 & 54.4 & 70.3 & 48.1 & 60.3 \\
                        Ours: Base & 65.9 & 45.6 & 57.2 & 69.4 & 50.8 & 61.3 \\
                        Ours: Base (OD_INP) & 65.3 & 40.3 &

In [68]:
def get_results(task, ov, od_inp):
    for run in recent_runs:
        if (run.config["task"] == task) & (run.config["overlap"] == ov) & (run.config["inpainting_old_od"] == od_inp):
            if run.state == "finished":
                if run.summary["Final/MeanIoU_All"] == "X":
                    continue
                else:
                    final_miou = np.round(float(run.summary["Final/MeanIoU_All"]) * 100, 2)
                    dense_miou = np.round(float(run.summary["Final/MeanIoU_Dense"]) * 100, 2)
                    incr_miou = np.round(float(run.summary["Final/MeanIoU_Incr"]) * 100, 2)
                    print(run.name)
                    print(f"& {dense_miou} & {incr_miou} & {final_miou} & - ")
    return None

In [76]:
#           task    overlap   od_inp
get_results("10-1", True, True)

voc-10-1-ov_Incr_Gen_Lora_OD_Inp_RR
& 46.23 & 13.7 & 29.78 & - 
